In [1]:
import numpy as np
from numpy.linalg import inv
from math import log

In [2]:
class SMO:
    def __init__(self, time_step):
        self.history_smo = [0]
        self.total_count = 0
        self.time = 0
        self.step_disc = time_step
        self.cur_state = 0
        self.history_smo_disc = [0]
        self.rejects = 0
        self.history = [0]
        self.history_disc = [0]
        self.states = [1, 0, 0]

    def get_delta(self, intensity):
        return - 1 / intensity * log(np.random.uniform(0, 1))

    def event(self, deltatime):
        self.time += deltatime
        occurences = int(self.time / self.step_disc) - len(self.history) + 1
        suffix = [self.cur_state] * occurences
        self.history_smo += suffix
        for _ in range(occurences):
            last_time = self.history[-1]
            self.history.append(last_time + self.step_disc)
        self.history_disc.append(self.time)

In [3]:
max_time = 5000
X = 6
t_obs = 1 / 3
t_r = 1 / 2
step = 1

smo = SMO(step)

print(f'Параметры X={X}, t_obs={t_obs}, t_r={t_r} с шагом дискредитации {step}')

Параметры X=6, t_obs=0.3333333333333333, t_r=0.5 с шагом дискредитации 1


In [4]:
def perform_rejects(delta):
    buffer_time = smo.get_delta(X)
    while buffer_time < delta:
        smo.rejects += 1
        smo.total_count += 1
        buffer_time += smo.get_delta(X)


while smo.time < max_time:
    if smo.cur_state == 0:
        delta_to_2 = smo.get_delta(0.9 / t_obs)
        delta_to_1 = smo.get_delta(X)
        if delta_to_2 >= delta_to_1:
            smo.event(delta_to_1)
            smo.cur_state = 1
            smo.total_count += 1
        else:
            smo.event(delta_to_2)
            smo.cur_state = 2
            perform_rejects(delta_to_2)

    elif smo.cur_state == 1:
        delta_to_0 = smo.get_delta(1 / t_obs)
        delta_to_2 = smo.get_delta(X)
        if delta_to_2 >= delta_to_0:
            smo.event(delta_to_0)
            smo.cur_state = 0
            perform_rejects(delta_to_0)
        else:
            smo.event(delta_to_2)
            smo.cur_state = 2

            perform_rejects(delta_to_2)
    elif smo.cur_state == 2:
        delta_to_0 = smo.get_delta(1 / t_r)
        smo.event(delta_to_0)
        smo.cur_state = 0

        perform_rejects(delta_to_0)
    else:
        raise Exception('!!!')
    smo.states[smo.cur_state] += 1
    smo.history_smo_disc.append(smo.cur_state)


def get_items(item, param_list):
    count = 0
    for element in param_list:
        if element == item:
            count += 1
    return count

In [5]:
print('-- Эмпирические характеристики --')
state_count = len(smo.history_smo)
print(f'1) Финальная вероятность p0 = {get_items(0, smo.history_smo) / state_count}')
print(f'2) Финальная вероятность p1 = {get_items(1, smo.history_smo) / state_count}')
print(f'3) Финальная вероятность p2 = {get_items(2, smo.history_smo) / state_count}')
print(f'4) Абсолютная пропускная спопобность = {(smo.total_count - smo.rejects) / smo.time}')
print(f'5) Относительная пропускная способность = {(smo.total_count - smo.rejects) / smo.total_count}')

print('_________________________________________________________________________')

print('-- Теоретические характеристики --')
p = inv(np.array([
    [-X - 0.9 / t_obs, 1 / t_obs, 1 / t_r],
    [X, -X - 1 / t_obs, 0],
    [1, 1, 1]
])).dot(np.array([[0], [0], [1]]))
print(f'1) Финальная вероятность p0 = {p[0]}')
print(f'2) Финальная вероятность p1 = {p[1]}')
print(f'3) Финальная вероятность p2 = {p[2]}')
print(f'4) Абсолютная пропускная спопобность = {X * p[0]}')
print(f'5) Относительная пропускная способность = {p[0]}')

-- Эмпирические характеристики --
1) Финальная вероятность p0 = 0.20975804839032194
2) Финальная вероятность p1 = 0.13097380523895222
3) Финальная вероятность p2 = 0.6592681463707258
4) Абсолютная пропускная спопобность = 1.2363147131933072
5) Относительная пропускная способность = 0.1951512090409748
_________________________________________________________________________
-- Теоретические характеристики --
1) Финальная вероятность p0 = [0.19933555]
2) Финальная вероятность p1 = [0.13289037]
3) Финальная вероятность p2 = [0.66777409]
4) Абсолютная пропускная спопобность = [1.19601329]
5) Относительная пропускная способность = [0.19933555]
